In [4]:
import numpy as np
from exputils.RoM.custom import calculate_RoM_custom
from exputils.actual_Amat import get_actual_Amat
from exputils.dot.get_topK_botK_Amat import get_topK_botK_Amat
from exputils.state.random import make_random_quantum_state
from exputils.RoM.actual import calculate_RoM_actual
from exputils.RoM.dot import calculate_RoM_dot
import random
import scipy.sparse

random.seed(0)
np.random.seed(0)


def calculate_RoM_CG(
    n_qubit: int,
    _K: float,
    rho_vec: np.ndarray,
    method: str = "gurobi",
    verbose: bool = False,
    iter_max: int = 100,
    eps: float = 1e-5,
    discard_current_threshold: float = 0.9,
):
    K = [1, 1, 1, 1, 0.1, 0.01, 0.001, 0.00001][n_qubit]
    assert K == _K

    current_Amat = get_actual_Amat(1)
    for _ in range(n_qubit - 1):
        current_Amat = scipy.sparse.kron(current_Amat, get_actual_Amat(1), format="csc")

    RoM_hist = []
    violations_hist = []

    for it in range(iter_max):
        print(
            f"iteration: {it + 1} / {iter_max}, # of columns = {current_Amat.shape[1]}"
        )
        RoM, coeff, dual = calculate_RoM_custom(
            current_Amat,
            rho_vec,
            method=method,
            return_dual=True,
            crossover=False,
            presolve=False,
            verbose=verbose,
        )
        print(f"{RoM = }")

        RoM_hist.append(RoM)

        dual_Amat = get_topK_botK_Amat(n_qubit, dual, K, verbose=False)
        dual_dots = np.abs(dual.T @ dual_Amat)
        dual_violated_indices = dual_dots > 1 + eps
        violated_count = np.sum(dual_violated_indices)
        violations_hist.append(violated_count)
        print(
            "# of violations:",
            f"{violated_count}"
            if violated_count < dual_Amat.shape[1]
            else f"more than {violated_count}",
        )

        nonbasic_indices = np.abs(coeff) > eps
        critical_indices = np.abs(dual @ current_Amat) >= (
            discard_current_threshold - eps
        )
        remain_indices = np.logical_or(nonbasic_indices, critical_indices)
        current_Amat = current_Amat[:, remain_indices]

        if violated_count == 0:
            print("exact RoM found!")
            break
        else:
            indices = np.where(dual_violated_indices)[0]
            extra_Amat = dual_Amat[:, indices]

        current_Amat = scipy.sparse.hstack((current_Amat, extra_Amat))

    assert len(RoM_hist) == len(violations_hist)
    print(f"{RoM_hist=}")
    print(f"{violations_hist=}")

    return RoM


n = 6
K = 0.001

RoM_sub_mul_list = []
RoM_list = []

for trial in range(10):
    rho = np.ones(1)
    RoM_sub_mul = 1.0
    for i in range(n):
        rho_i = make_random_quantum_state("mixed", 1, random.randint(0, 1000))
        RoM_i = calculate_RoM_actual(1, rho_i, method="gurobi")[0]
        print(f"{RoM_i=:0.5f}")
        RoM_sub_mul *= RoM_i
        rho = np.kron(rho, rho_i)
    print(f"{RoM_sub_mul=:0.5f}")
    assert rho.shape == (4**n,)
    RoM = calculate_RoM_CG(n, K, rho_vec=rho, verbose=True)
    print(f"{RoM=:0.5f}")
    RoM_sub_mul_list.append(RoM_sub_mul)
    RoM_list.append(RoM)

print(f"{RoM_sub_mul_list=}")
print(f"{RoM_list=}")

RoM_i=1.56313
RoM_i=1.51571
RoM_i=1.55121
RoM_i=1.36239
RoM_i=1.00000
RoM_i=1.20081
RoM_sub_mul=6.01256
iteration: 1 / 100, # of columns = 46656
Set parameter Method to value 2
Set parameter Crossover to value 0
Set parameter Presolve to value 0
Gurobi Optimizer version 10.0.1 build v10.0.1rc0 (win64)

CPU model: 11th Gen Intel(R) Core(TM) i5-1135G7 @ 2.40GHz, instruction set [SSE2|AVX|AVX2|AVX512]
Thread count: 4 physical cores, 8 logical processors, using up to 8 threads

Optimize a model with 4096 rows, 93312 columns and 5971968 nonzeros
Model fingerprint: 0xf96ac23e
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [1e+00, 1e+00]
  Bounds range     [0e+00, 0e+00]
  RHS range        [9e-05, 1e+00]
Ordering time: 0.10s

Barrier statistics:
 AA' NZ     : 4.980e+05
 Factor NZ  : 2.283e+06 (roughly 60 MB of memory)
 Factor Ops : 1.906e+09 (less than 1 second per iteration)
 Threads    : 4

                  Objective                Residual
Iter       Primal  

In [5]:
import numpy as np


RoM_sub_mul_list = [
    6.012555968675061,
    3.575627259319668,
    2.28545052608786,
    1.661495886288932,
    2.1809603987242934,
    2.586247149752913,
    1.8614635828673822,
    3.448883612819365,
    3.124994242354022,
    2.289510103188186,
]
RoM_list = [
    3.2445722576415856,
    2.4024482623243566,
    1.9009295937843866,
    1.575912920484231,
    1.8213201574405762,
    2.004490776836264,
    1.5651094078346874,
    2.328221025464809,
    2.2976762274599944,
    1.8726940202377833,
]

max_data = (-np.inf, None, None)
min_data = (+np.inf, None, None)

for sub_mul, RoM in zip(RoM_sub_mul_list, RoM_list):
    print(sub_mul, RoM, sub_mul - RoM)
    if sub_mul - RoM > max_data[0]:
        max_data = (sub_mul - RoM, sub_mul, RoM)
    if sub_mul - RoM < min_data[0]:
        min_data = (sub_mul - RoM, sub_mul, RoM)

print(f"diff:{max_data[0]:.3f} sub_mul:{max_data[1]:.3f} RoM:{max_data[2]:.3f}")
print(f"diff:{min_data[0]:.3f} sub_mul:{min_data[1]:.3f} RoM:{min_data[2]:.3f}")

6.012555968675061 3.2445722576415856 2.7679837110334753
3.575627259319668 2.4024482623243566 1.1731789969953113
2.28545052608786 1.9009295937843866 0.3845209323034733
1.661495886288932 1.575912920484231 0.08558296580470093
2.1809603987242934 1.8213201574405762 0.3596402412837172
2.586247149752913 2.004490776836264 0.5817563729166491
1.8614635828673822 1.5651094078346874 0.2963541750326948
3.448883612819365 2.328221025464809 1.120662587354556
3.124994242354022 2.2976762274599944 0.8273180148940273
2.289510103188186 1.8726940202377833 0.4168160829504026
diff:2.768 sub_mul:6.013 RoM:3.245
diff:0.086 sub_mul:1.661 RoM:1.576
